Validation and Robustness Testing

In [1]:
import numpy as np
import pandas as pd

def calculate_sharpe_ratio(returns):
    """Calculate Sharpe ratio."""
    if returns.std() == 0:
        return np.nan
    return returns.mean() / returns.std() * np.sqrt(252)  # Annualized

def calculate_max_drawdown(returns):
    """Calculate maximum drawdown."""
    cumulative = (1 + returns).cumprod()
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max
    return drawdown.min()

def run_monte_carlo_simulation(strategy_returns, simulations=10000):
    """Performs a bootstrap analysis on trade returns."""
    # We bootstrap daily returns for simplicity, trade-level is also valid
    daily_returns = strategy_returns[strategy_returns != 0]
    
    if len(daily_returns) == 0:
        print("No non-zero returns to bootstrap from!")
        return
    
    sim_results = []
    
    for _ in range(simulations):
        # Sample with replacement
        synthetic_returns = np.random.choice(daily_returns.values, size=len(strategy_returns), replace=True)
        synthetic_returns = pd.Series(synthetic_returns, index=strategy_returns.index)
        
        # Calculate metrics using our custom functions
        sharpe = calculate_sharpe_ratio(synthetic_returns)
        max_dd = calculate_max_drawdown(synthetic_returns)
        
        sim_results.append({'sharpe': sharpe, 'max_drawdown': max_dd})
        
    results_df = pd.DataFrame(sim_results)
    
    # Calculate original metrics
    original_sharpe = calculate_sharpe_ratio(strategy_returns)
    original_max_dd = calculate_max_drawdown(strategy_returns)
    
    print("\n--- Monte Carlo Simulation Results ---")
    print(f"Original Sharpe Ratio: {original_sharpe:.2f}")
    print(f"5th Percentile Sharpe: {results_df['sharpe'].quantile(0.05):.2f}")
    print(f"95th Percentile Sharpe: {results_df['sharpe'].quantile(0.95):.2f}")
    
    print(f"\nOriginal Max Drawdown: {original_max_dd:.2%}")
    print(f"95th Percentile (worse) Drawdown: {results_df['max_drawdown'].quantile(0.95):.2%}")
    print(f"5th Percentile (better) Drawdown: {results_df['max_drawdown'].quantile(0.05):.2%}")
    
    # Calculate p-values for significance
    sharpe_p_value = (results_df['sharpe'] > original_sharpe).mean()
    print(f"\nP-value for Sharpe Ratio (strategy > 0): {sharpe_p_value:.3f}")
    
    return results_df

# Example usage:
if __name__ == '__main__':
    # For testing purposes, create some sample returns
    np.random.seed(42)
    sample_returns = pd.Series(np.random.normal(0.0005, 0.02, 1000))  # Small positive daily returns
    
    # Add some actual trading pattern to make it more realistic
    sample_returns.iloc[::50] = np.random.normal(-0.05, 0.01, len(sample_returns[::50]))  # Occasional losses
    
    run_monte_carlo_simulation(sample_returns)


--- Monte Carlo Simulation Results ---
Original Sharpe Ratio: -0.02
5th Percentile Sharpe: -0.84
95th Percentile Sharpe: 0.83

Original Max Drawdown: -59.72%
95th Percentile (worse) Drawdown: -33.64%
5th Percentile (better) Drawdown: -79.90%

P-value for Sharpe Ratio (strategy > 0): 0.501
